Bir RBM oluşturup kullanıcıların bir filmi beğenip beğenmeyeceğini tahmin ettireceğiz.

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [2]:
# importing the dataset
# default seperatör ',' oluyor ama film isimleri virgünl içerebildiği için farklı seperator kullanıyoruz
# sütun başlıkları olmadığı için None, bazı film isimleri utf-8'de olmayan özel karakterler içeriyor
movies = pd.read_csv("data/BM/ml-1m/movies.dat", sep = "::", header = None, engine = "python", encoding = "latin-1")
users = pd.read_csv("data/BM/ml-1m/users.dat", sep = "::", header = None, engine = "python", encoding = "latin-1")
ratings = pd.read_csv("data/BM/ml-1m/ratings.dat", sep = "::", header = None, engine = "python", encoding = "latin-1")
print("movie\'s id, name, contents", len(movies), "\n", movies[:3], "\n")
print("user's id, sex, age, job, zip code", len(users), "\n", users[:3], "\n")
print("user's id, movie's id, ratings(1-5), timesteps", len(ratings), "\n", ratings[:3], "\n")

movie's id, name, contents 3883 
    0                        1                             2
0  1         Toy Story (1995)   Animation|Children's|Comedy
1  2           Jumanji (1995)  Adventure|Children's|Fantasy
2  3  Grumpier Old Men (1995)                Comedy|Romance 

user's id, sex, age, job, zip code 6040 
    0  1   2   3      4
0  1  F   1  10  48067
1  2  M  56  16  70072
2  3  M  25  15  55117 

user's id, movie's id, ratings(1-5), timesteps 1000209 
    0     1  2          3
0  1  1193  5  978300760
1  1   661  3  978302109
2  1   914  3  978301968 



In [3]:
# preparing train and testset
# bu klasörde 100k veri k=5 fold cros validation için 5e bölünmüş durumda base=train test=test, ama sadece birini kullanacağız
training_set = pd.read_csv("data/BM/ml-100k/u1.base", delimiter = "\t")
training_set = np.array(training_set, dtype = "int")
test_set = pd.read_csv("data/BM/ml-100k/u1.test", delimiter = "\t")
test_set = np.array(test_set, dtype = "int")
print(len(training_set), len(test_set))
print(training_set[:3])
print(test_set[:3]) # yukarıdaki ratings'in aynısı

79999 19999
[[        1         2         3 876893171]
 [        1         3         4 878542960]
 [        1         4         3 876893119]]
[[        1        10         3 875693118]
 [        1        12         5 878542960]
 [        1        14         5 874965706]]


In [4]:
# Kişiler için matrisler oluşturacağız hangi tarz filmleri beğendiklerine yönelik, max id alsak users sayısını buluruz
nb_users = max(max(training_set[:, 0]), max(test_set[:, 0]))
nb_movies = max(max(training_set[:, 1]), max(test_set[:, 1]))
print(nb_users, nb_movies)

943 1682


In [5]:
# BM matris verisi istediği için users are rows & movies are columns olan matrixler oluşturuyoruz
def convert(data):
    new_data = list()
    for id_users in range(1, nb_users+1):
        id_movies = data[:, 1][data[:, 0] == id_users] # data[rows][condition], kullanıcının oyladığı filmleri aldık
        id_ratings = data[:, 2][data[:, 0] == id_users] # puanlamadığı filmler 0'lar ile doldurulacak
        ratings = np.zeros(nb_movies)
        ratings[id_movies-1] = id_ratings
        new_data.append(list(ratings)) # torch expects list of lists
    return new_data

In [6]:
training_set = convert(training_set)
test_set = convert(test_set)
print(str(training_set[0])[:200])

[0.0, 3.0, 4.0, 3.0, 3.0, 0.0, 4.0, 1.0, 5.0, 0.0, 2.0, 0.0, 5.0, 0.0, 5.0, 5.0, 0.0, 4.0, 5.0, 0.0, 1.0, 4.0, 0.0, 0.0, 4.0, 3.0, 0.0, 4.0, 1.0, 3.0, 0.0, 5.0, 0.0, 2.0, 1.0, 0.0, 2.0, 3.0, 0.0, 3.0,


In [7]:
# Converting the data into Torch tensors
# Tensorler(multi dimensional matrix with single type) npArray'lerden daha etkili ve hızlı işlem yapılabilmesini sağlıyorlar
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)
print(training_set[0])

tensor([0., 3., 4.,  ..., 0., 0., 0.])


In [8]:
# Converting the ratings into binary ratings 1(like) or 0(not)
training_set[training_set == 0] = -1
training_set[training_set == 1] = 0 # tensorde OR yok, normalde var
training_set[training_set == 2] = 0
training_set[training_set >= 3] = 1

test_set[test_set == 0] = -1
test_set[test_set == 1] = 0 # tensorde OR yok, normalde var
test_set[test_set == 2] = 0
test_set[test_set >= 3] = 1
print(training_set[0])

tensor([-1.,  1.,  1.,  ..., -1., -1., -1.])


### Creating the Architecture of the RBM Network

In [59]:
class RBM():
    
    # nv number of visible nodes, nh number of hidden nodes
    def __init__(self, nv, nh):
        self.W = torch.randn(nh, nv) # initilize random weights
        self.a = torch.randn(1, nh) # bias for hidden nodes (2 boyutlu dizi ister), 1 = batch size
        self.b = torch.randn(1, nv) # bias for visible nodes
    
    def sample_h(self, x):
        wx = torch.mm(x, self.W.t()) # mm = tensorXtensor, t() = transpoz
        activation = wx + self.a.expand_as(wx) # bias is applied to every minibatch(1 yukarıda)
        # Probability of Hidden node is activated for Given Visible node
        p_h_given_v = torch.sigmoid(activation)
        return p_h_given_v, torch.bernoulli(p_h_given_v) # bernoulli(..) gives sampling of the hidden neurons
    
    def sample_v(self, y):
        wy = torch.mm(y, self.W)
        activation = wy + self.b.expand_as(wy)
        p_v_given_h = torch.sigmoid(activation)
        return p_v_given_h, torch.bernoulli(p_v_given_h)
    
    def train(self, v0, vk, ph0, phk):
        self.W += (torch.mm(v0.t(),ph0) - torch.mm(vk.t(),phk)).t() # first update
        self.b += torch.sum((v0 - vk), 0)
        self.a += torch.sum((ph0 - phk), 0)

### Training Phase

In [60]:
nv = len(training_set[0])
nh = 100 # num of features
batch_size = 100 # for fast training choose a high size
rbm = RBM(nv, nh)

In [61]:
nb_epoch = 5
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    counter = 0. # float
    for id_user in range(0, nb_users - batch_size, batch_size):
        vk = training_set[id_user : id_user + batch_size] # input batch of observations
        v0 = training_set[id_user : id_user + batch_size] # output batch of observations
        ph0,_ = rbm.sample_h(v0) # x,_ don't take the seccond returned value
        for k in range(10):
            _,hk = rbm.sample_h(vk)
            _,vk = rbm.sample_v(hk)
            vk[v0 < 0] = v0[v0 < 0]
        phk,_ = rbm.sample_h(vk)
        rbm.train(v0, vk, ph0, phk)
        # v0 değişmeyen visible nodelar, vk güncellenen nodelar son halleri
        train_loss += torch.mean(torch.abs(v0[v0 >= 0] - vk[v0 >= 0]))
        counter += 1.
    print("Epoch: " + str(epoch) + " Loss: " + str(train_loss / counter))

Epoch: 1 Loss: tensor(0.3309)
Epoch: 2 Loss: tensor(0.2477)
Epoch: 3 Loss: tensor(0.2499)
Epoch: 4 Loss: tensor(0.2504)
Epoch: 5 Loss: tensor(0.2469)


### Testing the RBM
Markov Chain Monte Carlo (blind walk) technique

In [62]:
test_loss = 0
counter = 0.
for id_user in range(nb_users): # test with each user
    v = training_set[id_user : id_user + 1]
    vt = test_set[id_user : id_user + 1] # target
    if len(vt[vt >= 0]) > 0: # -1 olan oylamaları kabul etmiyoruz zaten oylanmamış demekti
        _,h = rbm.sample_h(v)
        _,v = rbm.sample_v(h)
        test_loss += torch.mean(torch.abs(vt[vt >= 0] - v[vt >= 0]))
        counter += 1.0
print("Test loss: " + str(test_loss / counter))

Test loss: tensor(0.2581)


sınıfa predict için eklenebilir<br><br>

def predict( self, x): # x: visible nodes<br>
  _, h = self.sample_h( x)<br>
  _, v = self.sample_v( h)<br>
  return v